In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException

# Configure Chrome options to disable loading CSS
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")
chrome_options.add_argument("--disable-features=NetworkService")
chrome_options.add_argument("--dns-prefetch-disable")
chrome_options.add_argument("--disable-features=VizDisplayCompositor")

# Create a WebDriver instance
driver = webdriver.Chrome(options=chrome_options)

# Define the team_code_name_map
team_code_name_map = {
    'CSK': 'chennai-super-kings',
    'RR': 'rajasthan-royals',
    'RCB': 'royal-challengers-bangalore',
    'DC': 'delhi-capitals',
    'LSG': 'lucknow-super-giants',
    'MI': 'mumbai-indians',
    'GT': 'gujarat-titans',
    'PBKS': 'punjab-kings',
    'KKR': 'kolkata-knight-riders',
    'SRH': 'sunrisers-hyderabad'
}

# Dictionary to store player links for each team
player_links = {}

# URL base
url_base = "https://www.iplt20.com/teams/"

# Iterate over team names and construct URLs
for team_code, team_name in team_code_name_map.items():
    url = url_base + team_name
    driver.get(url)
    
    # Wait for the page to load completely
    wait = WebDriverWait(driver, 10)
    
    try:
        # Find all 'a' tags with the attribute 'data-player_name'
        player_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[data-player_name]')))
        
        # Extract href attribute values
        player_links[team_code] = [element.get_attribute('href') for element in player_elements]
    
    except StaleElementReferenceException:
        # If element is stale, re-locate and try again
        player_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[data-player_name]')))
        player_links[team_code] = [element.get_attribute('href') for element in player_elements]

# Print player links for each team
for team_code, links in player_links.items():
    print(f"{team_code}: {links}")

# Quit the WebDriver
driver.quit()


CSK: ['https://www.iplt20.com/teams/chennai-super-kings/squad-details/5443', 'https://www.iplt20.com/teams/chennai-super-kings/squad-details/20572', 'https://www.iplt20.com/teams/chennai-super-kings/squad-details/1', 'https://www.iplt20.com/teams/chennai-super-kings/squad-details/135', 'https://www.iplt20.com/teams/chennai-super-kings/squad-details/5612', 'https://www.iplt20.com/teams/chennai-super-kings/squad-details/20689', 'https://www.iplt20.com/teams/chennai-super-kings/squad-details/20715', 'https://www.iplt20.com/teams/chennai-super-kings/squad-details/9', 'https://www.iplt20.com/teams/chennai-super-kings/squad-details/1903', 'https://www.iplt20.com/teams/chennai-super-kings/squad-details/1735', 'https://www.iplt20.com/teams/chennai-super-kings/squad-details/5431', 'https://www.iplt20.com/teams/chennai-super-kings/squad-details/5615', 'https://www.iplt20.com/teams/chennai-super-kings/squad-details/5811', 'https://www.iplt20.com/teams/chennai-super-kings/squad-details/20684', 'ht

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")
chrome_options.add_argument("--disable-features=NetworkService")
chrome_options.add_argument("--dns-prefetch-disable")
chrome_options.add_argument("--disable-features=VizDisplayCompositor")

# Create a WebDriver instance
driver = webdriver.Chrome(options=chrome_options)

# URL of the page
base_url = "https://www.iplt20.com/teams/{}/squad-details/{}"

# Initialize player stats map
player_stats_map = {}

# Initialize player name map
player_name_map = {}

# Iterate through each team's player links
for team_code, links in player_links.items():

    # Initialize team's player stats dictionary
    player_stats_map[team_code] = {}
    
    # Iterate through each player link
    for link in links:
        # Extract player id from the link
        player_id = link.split("/")[-1]
        
        # Construct the player's squad details URL
        squad_details_url = base_url.format(team_code_name_map[team_code], player_id)
        
        # Load the squad details page
        driver.get(squad_details_url)
        
        # Wait for the player name to be present
        wait = WebDriverWait(driver, 10)
        player_name_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".plyr-name-nationality h1")))
        
        # Extract the player's name
        player_name = player_name_element.text.strip()
        
        # Add player name to player name map
        player_name_map[player_id] = player_name
        
        # Wait for the tables to be present
        tables = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "sm-pp-table")))
        
        # Initialize batting and bowling stats dictionaries
        batting_stats = {}
        bowling_stats = {}
        
        # Process each table
        for i, table in enumerate(tables):
            # Find and extract column values from the table
            column_values = []
            thead = table.find_element(By.TAG_NAME, "thead")
            th_tags = thead.find_elements(By.TAG_NAME, "th")
            for th_tag in th_tags:
                column_values.append(th_tag.text.strip())
            
            # Find and extract row values from the table
            row_values = []
            tbody = table.find_element(By.TAG_NAME, "tbody")
            tr_tags = tbody.find_elements(By.TAG_NAME, "tr")
            for tr_tag in tr_tags:
                td_tags = tr_tag.find_elements(By.TAG_NAME, "td")
                row_values.append([td_tag.text.strip() for td_tag in td_tags])
            
            # Map column values with row values
            if i == 0:  # First table for Batting stats
                batting_stats= {}
                for col, values in zip(column_values, zip(*row_values)):
                    batting_stats[col] = list(values)
            elif i == 1:  # Second table for Bowling stats
                bowling_stats = {}
                for col, values in zip(column_values, zip(*row_values)):
                    bowling_stats[col] = list(values)
        
        # Add batting and bowling stats to player stats map
        player_stats_map[team_code][player_id] = {
            "name": player_name,
            "batting_stats": batting_stats,
            "bowling_stats": bowling_stats
        }

print(player_name_map)

print(player_stats_map)

# Quit the WebDriver
driver.quit()


{'5443': 'Ruturaj Gaikwad', '20572': 'Devon Conway', '1': 'MS Dhoni', '135': 'Ajinkya Rahane', '5612': 'Shaik Rasheed', '20689': 'Sameer Rizvi', '20715': 'Avanish Rao Aravelly', '9': 'Ravindra Jadeja', '1903': 'Mitchell Santner', '1735': 'Moeen Ali', '5431': 'Shivam Dube', '5615': 'Nishant Sindhu', '5811': 'Ajay Mandal', '20684': 'Rachin Ravindra', '1745': 'Shardul Thakur', '20617': 'Daryl Mitchell', '20730': 'Richard Gleeson', '20571': 'Rajvardhan Hangargekar', '140': 'Deepak Chahar', '20570': 'Maheesh Theekshana', '20575': 'Mukesh Choudhary', '1594': 'Mustafizur Rahman', '20576': 'Prashant Solanki', '10789': 'Simarjeet Singh', '3257': 'Tushar Deshpande', '20627': 'Matheesha Pathirana', '258': 'Sanju Samson', '509': 'Jos Buttler', '1705': 'Shimron Hetmyer', '13538': 'Yashasvi Jaiswal', '20620': 'Dhruv Jurel', '4445': 'Riyan Parag', '5763': 'Kunal Rathore', '5652': 'Donovan Ferreira', '3333': 'Rovman Powell', '20688': 'Shubham Dubey', '20697': 'Tom Kohler-Cadmore', '8': 'Ravichandran A

In [11]:
team_colors = {
    'PBKS' : 'D50F28',
    'DD' : '021346',
    'CSK' : 'EED201',
    'GL' : 'E25627',
    'MI' : '1230D6',
    'RCB' : 'C01F23',
    'KTK' : 'EB4917',
    'KKR' : '452858',
    'PWI' : '387A95',
    'DC' : '2937CE',
    'GT' : '3C4A63',
    'RPS' : 'DF4AAB',
    'RR' : 'DF3C65',
    'LSG' : '3C4874',
    'DEC' : '354D5E',
    'SRH' : 'FF5131',
}

import matplotlib.colors as mcolors


colors = {}
for team, hex_color in team_colors.items():
    rgba_color = mcolors.to_rgba('#' + hex_color)
    colors[team] = rgba_color

colors

{'PBKS': (0.8352941176470589, 0.058823529411764705, 0.1568627450980392, 1.0),
 'DD': (0.00784313725490196, 0.07450980392156863, 0.27450980392156865, 1.0),
 'CSK': (0.9333333333333333, 0.8235294117647058, 0.00392156862745098, 1.0),
 'GL': (0.8862745098039215, 0.33725490196078434, 0.15294117647058825, 1.0),
 'MI': (0.07058823529411765, 0.18823529411764706, 0.8392156862745098, 1.0),
 'RCB': (0.7529411764705882, 0.12156862745098039, 0.13725490196078433, 1.0),
 'KTK': (0.9215686274509803, 0.28627450980392155, 0.09019607843137255, 1.0),
 'KKR': (0.27058823529411763, 0.1568627450980392, 0.34509803921568627, 1.0),
 'PWI': (0.2196078431372549, 0.47843137254901963, 0.5843137254901961, 1.0),
 'DC': (0.1607843137254902, 0.21568627450980393, 0.807843137254902, 1.0),
 'GT': (0.23529411764705882, 0.2901960784313726, 0.38823529411764707, 1.0),
 'RPS': (0.8745098039215686, 0.2901960784313726, 0.6705882352941176, 1.0),
 'RR': (0.8745098039215686, 0.23529411764705882, 0.396078431372549, 1.0),
 'LSG': (0.

In [12]:
import csv

# Define the file name
file_name = 'ipl_2024_players_stats.csv'

# Initialize column names
columns = ['Player ID', 'Player Name', 'Team Name', 'Year', 'Matches', 'Not Outs', 'Runs', 'Highest Score', 'Average', 'Balls Faced', 'Strike Rate', 'Hundreds', 'Fifties', 'Fours', 'Sixes', 'Catches', 'Stumpings']

# Open the CSV file in write mode
with open(file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(columns)  # Write the header row
    
    # Iterate through each team
    for team_code in team_code_name_map.keys():
        # Iterate through each player
        for player_id, player_data in player_stats_map[team_code].items():
            player_name = player_data['name']
            team_name = team_code  # Get the team name from the team code
            batting_stats = player_data['batting_stats']
            
            # Write a row for each year's stats
            if 'YEAR' in batting_stats:
                if batting_stats['YEAR'][0] == 'No stats':
                    writer.writerow([player_id, player_name, team_name, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'])
                    continue

                for i in range(len(batting_stats['YEAR'])):
                    year = batting_stats['YEAR'][i]
                    matches = batting_stats.get('MAT', [])[i] if 'MAT' in batting_stats else '-'
                    not_outs = batting_stats.get('NO', [])[i] if 'NO' in batting_stats else '-'
                    runs = batting_stats.get('RUNS', [])[i] if 'RUNS' in batting_stats else '-'
                    highest_score = batting_stats.get('HS', [])[i] if 'HS' in batting_stats else '-'
                    average = batting_stats.get('AVG', [])[i] if 'AVG' in batting_stats else '-'
                    balls_faced = batting_stats.get('BF', [])[i] if 'BF' in batting_stats else '-'
                    strike_rate = batting_stats.get('SR', [])[i] if 'SR' in batting_stats else '-'
                    hundreds = batting_stats.get('100', [])[i] if '100' in batting_stats else '-'
                    fifties = batting_stats.get('50', [])[i] if '50' in batting_stats else '-'
                    fours = batting_stats.get('4S', [])[i] if '4S' in batting_stats else '-'
                    sixes = batting_stats.get('6S', [])[i] if '6S' in batting_stats else '-'
                    catches = batting_stats.get('CT', [])[i] if 'CT' in batting_stats else '-'
                    stumpings = batting_stats.get('ST', [])[i] if 'ST' in batting_stats else '-'
                    
                    writer.writerow([player_id, player_name, team_name, year, matches, not_outs, runs, highest_score, average, balls_faced, strike_rate, hundreds, fifties, fours, sixes, catches, stumpings])
            else:
                # Write a row with 'No stats' for each year if batting stats are missing
                for year in ['Career', '2024', '2023', '2022']:  # Assuming these are the common years
                    writer.writerow([player_id, player_name, team_name, year, 'No stats'] * (len(columns) - 3))

print(f'Data exported successfully to {file_name}')


Data exported successfully to ipl_2024_players_stats.csv


In [14]:
import csv

# Define the file name
file_name = 'ipl_2024_players_all_stats.csv'

# Initialize column names for batting and bowling stats
information_columns = ['Player ID', 'Player Name', 'Team Name', 'Year', 'Matches']
batting_columns = ['Not Outs', 'Runs', 'Highest Score', 'Average', 'Balls Faced', 'Strike Rate', 'Hundreds', 'Fifties', 'Fours', 'Sixes', 'Catches', 'Stumpings']
bowling_columns = ['Overs', 'Runs Given', 'Wickets', 'Best Bowling', 'Bowling Average', 'Bowling Economy', 'Bowling Strike Rate', '4 Wickets', '5 Wickets']

# Open the CSV file in write mode
with open(file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(information_columns + batting_columns + bowling_columns)  # Write the header row
    
    # Iterate through each team
    for team_code in team_code_name_map.keys():
        # Iterate through each player
        for player_id, player_data in player_stats_map[team_code].items():
            player_name = player_data['name']
            team_name = team_code  # Get the team name from the team code
            batting_stats = player_data['batting_stats']
            bowling_stats = player_data['bowling_stats']
            
            # Write a row for each year's stats
            if 'YEAR' in batting_stats:
                if batting_stats['YEAR'][0] == 'No stats':
                    writer.writerow([player_id, player_name, team_name, '-', '-'] + ['-'] * len(batting_columns) + ['-'] * len(bowling_columns))
                    continue

                for i in range(len(batting_stats['YEAR'])):
                    year = batting_stats['YEAR'][i]
                    matches = batting_stats.get('MAT', [])[i] if 'MAT' in batting_stats else '-'
                    not_outs = batting_stats.get('NO', [])[i] if 'NO' in batting_stats else '-'
                    runs = batting_stats.get('RUNS', [])[i] if 'RUNS' in batting_stats else '-'
                    highest_score = batting_stats.get('HS', [])[i] if 'HS' in batting_stats else '-'
                    average = batting_stats.get('AVG', [])[i] if 'AVG' in batting_stats else '-'
                    balls_faced = batting_stats.get('BF', [])[i] if 'BF' in batting_stats else '-'
                    strike_rate = batting_stats.get('SR', [])[i] if 'SR' in batting_stats else '-'
                    hundreds = batting_stats.get('100', [])[i] if '100' in batting_stats else '-'
                    fifties = batting_stats.get('50', [])[i] if '50' in batting_stats else '-'
                    fours = batting_stats.get('4S', [])[i] if '4S' in batting_stats else '-'
                    sixes = batting_stats.get('6S', [])[i] if '6S' in batting_stats else '-'
                    catches = batting_stats.get('CT', [])[i] if 'CT' in batting_stats else '-'
                    stumpings = batting_stats.get('ST', [])[i] if 'ST' in batting_stats else '-'
                    
                    # Get bowling stats for the corresponding year
                    bowling_year_index = bowling_stats['YEAR'].index(year) if year in bowling_stats['YEAR'] else None
                    if bowling_year_index is not None:
                        overs = bowling_stats.get('BALLS', [])[bowling_year_index] if 'BALLS' in bowling_stats else '-'
                        runs_given = bowling_stats.get('RUNS', [])[bowling_year_index] if 'RUNS' in bowling_stats else '-'
                        wickets = bowling_stats.get('WKTS', [])[bowling_year_index] if 'WKTS' in bowling_stats else '-'
                        best_bowling = bowling_stats.get('BBM', [])[bowling_year_index] if 'BBM' in bowling_stats else '-'
                        bowling_average = bowling_stats.get('AVE', [])[bowling_year_index] if 'AVE' in bowling_stats else '-'
                        bowling_economy = bowling_stats.get('ECON', [])[bowling_year_index] if 'ECON' in bowling_stats else '-'
                        bowling_strike_rate = bowling_stats.get('SR', [])[bowling_year_index] if 'SR' in bowling_stats else '-'
                        four_wickets = bowling_stats.get('4W', [])[bowling_year_index] if '4W' in bowling_stats else '-'
                        five_wickets = bowling_stats.get('5W', [])[bowling_year_index] if '5W' in bowling_stats else '-'
                    else:
                        # If bowling stats for the year are missing, fill with placeholders
                        overs = runs_given = wickets = best_bowling = bowling_average = bowling_economy = bowling_strike_rate = four_wickets = five_wickets = '-'
                    
                    # Replace empty fields with '-'
                    row = [player_id, player_name, team_name, year, matches, not_outs, runs, highest_score, average, balls_faced, strike_rate, hundreds, fifties, fours, sixes, catches, stumpings, overs, runs_given, wickets, best_bowling, bowling_average, bowling_economy, bowling_strike_rate, four_wickets, five_wickets]
                    row = ['-' if field == '' else field for field in row]
                    
                    writer.writerow(row)

print(f'Data exported successfully to {file_name}')


Data exported successfully to ipl_2024_players_all_stats.csv
